In [14]:
import matplotlib
import dotenv
from pathlib import Path
import os
import json

In [7]:
base_dir = Path.cwd()
data_dir = base_dir / "data"
ro_dir = data_dir / "judete"
mol_dir = data_dir / "raioane"

In [11]:
json_files = list(mol_dir.rglob('*.json'))

files = []
for file in json_files:
    files.append(file)
print(len(files))

6071


In [48]:
class DataSet:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.files = self.get_jsons()

    def get_jsons(self):
        content = []
        json_files = list(self.root_dir.rglob('*.json'))

        for path in json_files:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    content.append(data["content"])
            
            except json.JSONDecodeError:
                print(f"Skipping corrupt file: {path}")

        return content
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        return self.files[idx]
    
    def __add__(self, other):
        combined = DataSet(self.root_dir)
        combined.files = self.files + other.files
        return combined

merged_dataset = list(set(DataSet(mol_dir) + DataSet(ro_dir)))

In [49]:
print(len(merged_dataset))

20248


In [65]:
system_prompt = "You are an expert translator specializing in English"

prompt = """
You will translate Romanian and Moldovan text into English while preserving the original meaning and context. 
Ensure that the translations are accurate, culturally appropriate, and maintain the nuances of the source language.

Input_format: 
    <text>target_sentence1</text>
    <text>target_sentence2</text>
    ...
    <text>target_sentenceN</text>

Output_format: 
    <text>translated_sentence1</text>
    <text>translated_sentence2</text>
    ...
    <text>translated_sentenceN</text>

Respect the input and output formats strictly. 

Target sentences: 

{sentences}
"""

In [66]:
sample = merged_dataset[0:2]

formatted_prompt = ""
for i, sentence in enumerate(sample):
    formatted_prompt += f"<text>{sentence}</text>\n"

print(formatted_prompt)

<text>Într-un context marcat de narațiuni simplificate și propagandă anti-ucraineană, Asociația Publică „Arena Civilă” lansează proiectul media „Reportaje de pe linia frontului” , o inițiativă care aduce în prim-plan poveștile celor care trăiesc și muncesc în epicentrul conflictelor.

Scopul proiectului este de a evidenția dimensiunea umană a războiului, prin relatările jurnaliștilor militari din Ucraina, care, cu riscul propriei vieți, documentează adevărul din zonele de conflict. Evenimentele din cadrul proiectului vor avea loc în trei orașe ale Republicii Moldova: Chișinău, Bălți și Cahul .
• Chișinău : 26 noiembrie
• ora 13:00: Atelier „O mie de zile de război. Frontul și mass-media”

Roman Pahulich , jurnalist de război renumit și corespondent al proiectului „Donbas Realii” din cadrul Serviciului Ucrainean al Radio Europa Liberă, va fi invitatul special al evenimentului. Pahulich a realizat reportaje din cele mai fierbinți zone ale conflictului, inclusiv regiunile Sumî, Harkov, Do

In [67]:
def create_batches(dataset, batch_size):
    for i in range(0, len(dataset), batch_size):
        yield dataset.files[i:i + batch_size]

In [68]:
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash", system_instruction=system_prompt)

In [69]:
response = model.generate_content(formatted_prompt)
print(response.text)

**Text 1 Translation:**

In a context marked by simplified narratives and anti-Ukrainian propaganda, the Public Association "Civil Arena" is launching the media project "Reports from the Frontline," an initiative that highlights the stories of those living and working in the epicenter of conflicts.

The project aims to highlight the human dimension of the war through the accounts of military journalists from Ukraine who, at the risk of their own lives, document the truth from conflict zones. Events within the project will take place in three cities of the Republic of Moldova: Chișinău, Bălți, and Cahul.
• Chișinău: November 26
• 1:00 PM: Workshop "A Thousand Days of War. The Front and Mass Media"

Roman Pahulich, a renowned war journalist and correspondent for the "Donbas Realities" project within Radio Free Europe's Ukrainian Service, will be the special guest at the event. Pahulich has reported from the hottest conflict zones, including the Sumy, Kharkiv, Donetsk, and Zaporizhzhia re

In [43]:
x = list(response.text.splitlines())
print(len(x))

62
